In [0]:
import pandas as pd
import numpy as np 
import pickle
import gzip


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with open('/content/drive/My Drive/dl-ims/test_subjects','rb') as f:
               images = pickle.load(f)


#### Subject IDs

In [4]:
im_name_list =[]

for x1 in images:
    im_name_list.append(x1)
im_name_list

['OAS30001',
 'OAS30002',
 'OAS30004',
 'OAS30005',
 'OAS30006',
 'OAS30007',
 'OAS30008',
 'OAS30019',
 'OAS30022',
 'OAS30024',
 'OAS30027']

#### Making training and test sets

In [5]:
pixels=np.zeros((1, 120, 160, 120))
for name in im_name_list:
    pixels= np.append(pixels ,np.reshape(images[name]['image'], (1, 120, 160, 120) ), axis=0)

input_ims= pixels[1:12,:,:,:] 
input_ims.shape

(11, 120, 160, 120)

In [6]:
x_train = input_ims[0:9,:,:,:]
x_train.shape

(9, 120, 160, 120)

In [7]:
x_test=input_ims[9:12,:,:,:]
x_test.shape

(2, 120, 160, 120)

In [0]:
x_test= np.reshape(x_test, (2, 120, 160, 120, 1) )

#### Making target values

In [9]:
y_train= [images[key]['group'] for key in images.keys()]
y_test = y_train[-2:]
y_test

['AD', 'AD']

In [10]:
y_train = y_train[:9]
y_train

['CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'AD', 'AD']

In [11]:
def convert_numeric(x):
    if x == "AD":
        return 1
    elif x == "CN":
        return 0
    else:
        return x
    
    
y_train = list(map( convert_numeric, y_train))
y_train

[0, 0, 0, 0, 0, 0, 0, 1, 1]

In [12]:
y_test = list(map( convert_numeric, y_test))
y_test

[1, 1]

In [13]:
len(input_ims)

11

In [14]:
#looking into first image
first_im=images['OAS30001']
first = first_im['image']
first.shape

(120, 160, 120)

In [0]:
first_modified= np.reshape(first, (120, 160, 120 , 1))

In [0]:
x_train = np.reshape(x_train, (9, 120, 160, 120, 1) )

#### Building Encoder

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Conv3D, MaxPool3D, Dropout, Flatten, Conv3DTranspose, UpSampling3D, Reshape
from keras.layers import Input

Using TensorFlow backend.


In [18]:



input_img = Input(shape= (120, 160, 120 , 1))
x = Conv3D(10, (3,3,3), activation='relu', padding='same')(input_img)
x = MaxPool3D((2,2,2))(x)
x= Dropout(0.8)(x)

x = Conv3D(10, (3,3,3), activation='relu', padding='same')(x)
x = MaxPool3D((2,2,2))(x)
x= Dropout(0.8)(x)

x = Conv3D(10, (3,3,3), activation='relu', padding='same')(x)
x = MaxPool3D((2,2,2))(x)
x= Dropout(0.8)(x)

encoded_shape = K.int_shape(x)

x = Flatten()(x)

x = Dense(32, activation='selu')(x)
x= Dense(16, activation='selu')(x)
encoded = Dense(2, activation='selu')(x)


encoder=Model(input_img, encoded, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 120, 160, 120, 1)  0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 120, 160, 120, 10) 280       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 60, 80, 60, 10)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 80, 60, 10)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 60, 80, 60, 10)    2710      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 30, 40, 30, 10)    0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 40, 30, 10)    0   

#### Building decoder

In [19]:
encoded_input= Input(shape=(2,))
x = Dense(16, activation='selu')(encoded_input)
x= Dense(32, activation='selu')(x)
x= Dense(np.prod(encoded_shape[1:]))(x)
x= Reshape((encoded_shape[1], encoded_shape[2], encoded_shape[3], encoded_shape[4]))(x)

x = Conv3DTranspose(10, (3,3,3), activation='relu', padding='same')(x)
x = UpSampling3D((2,2,2))(x)
x= Dropout(0.8)(x)

x = Conv3DTranspose(10, (3,3,3), activation='relu', padding='same')(x)
x = UpSampling3D((2,2,2))(x)
x= Dropout(0.8)(x)

x = Conv3DTranspose(10, (3,3,3), activation='relu', padding='same')(x)
x = UpSampling3D((2,2,2))(x)
x= Dropout(0.8)(x)

x = Conv3DTranspose(1, (3,3,3), activation='sigmoid', padding='same')(x)

decoder = Model(encoded_input, x, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                48        
_________________________________________________________________
dense_5 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_6 (Dense)              (None, 45000)             1485000   
_________________________________________________________________
reshape_1 (Reshape)          (None, 15, 20, 15, 10)    0         
_________________________________________________________________
conv3d_transpose_1 (Conv3DTr (None, 15, 20, 15, 10)    2710      
_________________________________________________________________
up_sampling3d_1 (UpSampling3 (None, 30, 40, 30, 10)    0   

#### Building Autoencoder

In [20]:
autoencoder= Model(input_img, decoder(encoder(input_img)), name='autoencoder')
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 120, 160, 120, 1)  0         
_________________________________________________________________
encoder (Model)              (None, 2)                 1446294   
_________________________________________________________________
decoder (Model)              (None, 120, 160, 120, 1)  1493993   
Total params: 2,940,287
Trainable params: 2,940,287
Non-trainable params: 0
_________________________________________________________________


In [21]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

hist= autoencoder.fit(x_train, x_train, epochs=200, verbose=1, validation_data=(x_test, x_test))


Train on 9 samples, validate on 2 samples
Epoch 1/200


InternalError: ignored

#### Save Model

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()                       
drive = GoogleDrive(gauth)

In [0]:
#just to save the model. Don't run again
autoencoder.save('cae1.h5')
model_file = drive.CreateFile({'title' : 'cae1.h5'})                       
model_file.SetContentFile('cae1.h5')                       
model_file.Upload()

In [0]:
# Don't run again. download to google drive                       
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '18DNWPckBHblpR5KfaV7erHAZTLXvCBP5'})

In [0]:
#Loading models from Google Drive into Colab, Don't run

file_obj = drive.CreateFile({'id': '18DNWPckBHblpR5KfaV7erHAZTLXvCBP5'})                       
file_obj.GetContentFile('cae1.h5')


In [0]:
from keras.models import load_model
cae=load_model('/content/drive/My Drive/cae1.h5')

In [22]:
cae.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 120, 160, 120, 1)  0         
_________________________________________________________________
encoder (Model)              (None, 2)                 1446294   
_________________________________________________________________
decoder (Model)              (None, 120, 160, 120, 1)  1493993   
Total params: 2,940,287
Trainable params: 2,940,287
Non-trainable params: 0
_________________________________________________________________


In [80]:
cae.get_weights() #works

[array([[[[[ 1.07625946e-01,  5.98480888e-02, -1.45185012e-02,
             8.45264047e-02,  1.01154573e-01, -1.13513879e-01,
             3.19826938e-02, -5.00659794e-02,  3.53506347e-03,
            -9.46609750e-02]],
 
          [[-9.46612880e-02,  2.55575199e-02,  4.89623286e-02,
             2.10701674e-02, -4.44244780e-02,  5.98257184e-02,
             1.71006843e-02, -9.07185895e-04,  1.23221636e-01,
            -1.20552614e-01]],
 
          [[-7.69710839e-02, -1.20123260e-01, -1.24206915e-02,
            -9.87468660e-02, -7.58215711e-02,  6.39985129e-02,
            -6.75392002e-02,  9.52620655e-02, -9.62819830e-02,
            -1.09936416e-01]]],
 
 
         [[[-9.93242338e-02,  1.09578567e-02,  6.57714680e-02,
            -3.69692333e-02, -9.11215246e-02,  1.08695954e-01,
            -1.43012807e-01, -1.02111891e-01,  5.88062629e-02,
            -1.24878049e-01]],
 
          [[ 1.58766434e-02, -2.19105165e-02,  2.46413220e-02,
            -4.32057232e-02, -1.51453286e-01, 

In [0]:
encoder_weights = cae.layers[1].get_weights()

In [35]:
cae.layers[1].get_config()['layers'][1]

{'class_name': 'Conv3D',
 'config': {'activation': 'relu',
  'activity_regularizer': None,
  'bias_constraint': None,
  'bias_initializer': {'class_name': 'Zeros', 'config': {}},
  'bias_regularizer': None,
  'data_format': 'channels_last',
  'dilation_rate': (1, 1, 1),
  'dtype': 'float32',
  'filters': 10,
  'kernel_constraint': None,
  'kernel_initializer': {'class_name': 'VarianceScaling',
   'config': {'distribution': 'uniform',
    'mode': 'fan_avg',
    'scale': 1.0,
    'seed': None}},
  'kernel_regularizer': None,
  'kernel_size': (3, 3, 3),
  'name': 'conv3d_4',
  'padding': 'same',
  'strides': (1, 1, 1),
  'trainable': True,
  'use_bias': True},
 'inbound_nodes': [[['input_3', 0, 0, {}]]],
 'name': 'conv3d_4'}

In [0]:
encoder1 = cae.layers[1]

In [33]:
encoder1.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 120, 160, 120, 1)  0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 120, 160, 120, 10) 280       
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 60, 80, 60, 10)    0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 60, 80, 60, 10)    0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 60, 80, 60, 10)    2710      
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 30, 40, 30, 10)    0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 30, 40, 30, 10)    0   

In [34]:
encoder1.get_weights()

[array([[[[[ 1.07625946e-01,  5.98480888e-02, -1.45185012e-02,
             8.45264047e-02,  1.01154573e-01, -1.13513879e-01,
             3.19826938e-02, -5.00659794e-02,  3.53506347e-03,
            -9.46609750e-02]],
 
          [[-9.46612880e-02,  2.55575199e-02,  4.89623286e-02,
             2.10701674e-02, -4.44244780e-02,  5.98257184e-02,
             1.71006843e-02, -9.07185895e-04,  1.23221636e-01,
            -1.20552614e-01]],
 
          [[-7.69710839e-02, -1.20123260e-01, -1.24206915e-02,
            -9.87468660e-02, -7.58215711e-02,  6.39985129e-02,
            -6.75392002e-02,  9.52620655e-02, -9.62819830e-02,
            -1.09936416e-01]]],
 
 
         [[[-9.93242338e-02,  1.09578567e-02,  6.57714680e-02,
            -3.69692333e-02, -9.11215246e-02,  1.08695954e-01,
            -1.43012807e-01, -1.02111891e-01,  5.88062629e-02,
            -1.24878049e-01]],
 
          [[ 1.58766434e-02, -2.19105165e-02,  2.46413220e-02,
            -4.32057232e-02, -1.51453286e-01, 

In [0]:
#saving encoder, don't run again
encoder1.save('enc1.h5', include_optimizer=True)
enc = drive.CreateFile({'title' : 'enc1.h5'})                       
enc.SetContentFile('enc1.h5')                       
enc.Upload()

In [26]:
#load encoder

encoder = load_model('/content/drive/My Drive/enc1.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [27]:
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 120, 160, 120, 1)  0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 120, 160, 120, 10) 280       
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 60, 80, 60, 10)    0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 60, 80, 60, 10)    0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 60, 80, 60, 10)    2710      
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 30, 40, 30, 10)    0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 30, 40, 30, 10)    0   

In [31]:
encoder.get_weights()

[array([[[[[ 1.07625946e-01,  5.98480888e-02, -1.45185012e-02,
             8.45264047e-02,  1.01154573e-01, -1.13513879e-01,
             3.19826938e-02, -5.00659794e-02,  3.53506347e-03,
            -9.46609750e-02]],
 
          [[-9.46612880e-02,  2.55575199e-02,  4.89623286e-02,
             2.10701674e-02, -4.44244780e-02,  5.98257184e-02,
             1.71006843e-02, -9.07185895e-04,  1.23221636e-01,
            -1.20552614e-01]],
 
          [[-7.69710839e-02, -1.20123260e-01, -1.24206915e-02,
            -9.87468660e-02, -7.58215711e-02,  6.39985129e-02,
            -6.75392002e-02,  9.52620655e-02, -9.62819830e-02,
            -1.09936416e-01]]],
 
 
         [[[-9.93242338e-02,  1.09578567e-02,  6.57714680e-02,
            -3.69692333e-02, -9.11215246e-02,  1.08695954e-01,
            -1.43012807e-01, -1.02111891e-01,  5.88062629e-02,
            -1.24878049e-01]],
 
          [[ 1.58766434e-02, -2.19105165e-02,  2.46413220e-02,
            -4.32057232e-02, -1.51453286e-01, 

In [69]:
encoder.layers[1].get_weights()

[array([[[[[ 1.07625946e-01,  5.98480888e-02, -1.45185012e-02,
             8.45264047e-02,  1.01154573e-01, -1.13513879e-01,
             3.19826938e-02, -5.00659794e-02,  3.53506347e-03,
            -9.46609750e-02]],
 
          [[-9.46612880e-02,  2.55575199e-02,  4.89623286e-02,
             2.10701674e-02, -4.44244780e-02,  5.98257184e-02,
             1.71006843e-02, -9.07185895e-04,  1.23221636e-01,
            -1.20552614e-01]],
 
          [[-7.69710839e-02, -1.20123260e-01, -1.24206915e-02,
            -9.87468660e-02, -7.58215711e-02,  6.39985129e-02,
            -6.75392002e-02,  9.52620655e-02, -9.62819830e-02,
            -1.09936416e-01]]],
 
 
         [[[-9.93242338e-02,  1.09578567e-02,  6.57714680e-02,
            -3.69692333e-02, -9.11215246e-02,  1.08695954e-01,
            -1.43012807e-01, -1.02111891e-01,  5.88062629e-02,
            -1.24878049e-01]],
 
          [[ 1.58766434e-02, -2.19105165e-02,  2.46413220e-02,
            -4.32057232e-02, -1.51453286e-01, 

In [0]:
!pip install -q pyyaml h5py

In [60]:
# Don't run again. download to google drive                       
drive.CreateFile({'id': enc.get('id')})

GoogleDriveFile({'id': '1E-CGCjUDvkmmpcRrIhawIIpWKCDtPpgu'})

In [0]:
#Loading models from Google Drive into Colab, Don't run

file_enc = drive.CreateFile({'id': '1E-CGCjUDvkmmpcRrIhawIIpWKCDtPpgu'})                       
file_enc.GetContentFile('enc1.h5')

In [0]:
encoder.set_weights(encoder_weights)

In [70]:
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 120, 160, 120, 1)  0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 120, 160, 120, 10) 280       
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 60, 80, 60, 10)    0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 60, 80, 60, 10)    0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 60, 80, 60, 10)    2710      
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 30, 40, 30, 10)    0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 30, 40, 30, 10)    0   

In [74]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten
from keras.models import Model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Conv3D, MaxPool3D, Dropout, Flatten, Conv3DTranspose, UpSampling3D, Reshape
from keras.layers import Input
from keras.layers import *
# First, define the vision modules
digit_input = Input(shape=(27, 27, 1))
x = Conv2D(64, (3, 3))(digit_input)
x = Conv2D(64, (3, 3))(x)
x = MaxPooling2D((2, 2))(x)
out = Flatten()(x)

vision_model = Model(digit_input, out)

# Then define the tell-digits-apart model
digit_a = Input(shape=(27, 27, 1))
digit_b = Input(shape=(27, 27, 1))

# The vision model will be shared, weights and all
out_a = vision_model(digit_a)
out_b = vision_model(digit_b)

concatenated = keras.layers.concatenate([out_a, out_b])
out = Dense(1, activation='sigmoid')(concatenated)

classification_model = Model([digit_a, digit_b], out)

AttributeError: ignored

In [73]:
keras.layers.concatenate([out_a, out_b])

AttributeError: ignored

In [75]:
from keras.layers import Conv2D, Input

# input tensor for a 3-channel 256x256 image
x = Input(shape=(256, 256, 3))
# 3x3 conv with 3 output channels (same as input channels)
y = Conv2D(3, (3, 3), padding='same')(x)
# this returns x + y.
z = keras.layers.add([x, y])

AttributeError: ignored

In [84]:
encoder.add(Activation('softmax'))

AttributeError: ignored

#### Supervised Learning

In [0]:

input_img = Input(shape= (120, 160, 120 , 1))
x = Conv3D(10, (3,3,3), activation='relu', padding='same')(input_img)
x = MaxPool3D((2,2,2))(x)
x= Dropout(0.8)(x)

x = Conv3D(10, (3,3,3), activation='relu', padding='same')(x)
x = MaxPool3D((2,2,2))(x)
x= Dropout(0.8)(x)

x = Conv3D(10, (3,3,3), activation='relu', padding='same')(x)
x = MaxPool3D((2,2,2))(x)
x= Dropout(0.8)(x)

encoded_shape = K.int_shape(x)

x = Flatten()(x)

x = Dense(32, activation='selu')(x)
x= Dense(16, activation='selu')(x)
encoded = Dense(2, activation='selu')(x)


supervised_encoder=Model(input_img, encoded, name='encoder')
supervised_encoder.summary()

In [0]:
supervised_encoder.set_weights(encoder_weights)

In [0]:
supervised_encoder.compile(optimizer='adam', loss='binary_crossentropy') 
#add l1 and l2 regularizer, fine tune, add learning rate, momentum

In [0]:
sup_hist = supervised_encoder.fit(x_train, y , batch_size=10, verbose=1 , validation_split=0.3)